In [13]:
# ニューラルネットワークを使って予測してみる

import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.base import ClassifierMixin, BaseEstimator, is_classifier
from sklearn.utils.validation import check_is_fitted
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from scipy.stats import uniform, randint
from lightgbm import LGBMClassifier

# データの読み込み
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# データの確認
display(train.dtypes)
display(test.dtypes)

# データ全体の確認
display(train.describe())
display(test.describe())

# 欠損データの確認
kesson_train = train.isnull().sum()
kesson_test = test.isnull().sum()
display(kesson_train)
display(kesson_test)


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [17]:
# 名前から敬称を抽出、家族サイズの作成、AgeとFareをカテゴリ化する関数
def feature_engineering(df):
    # 敬称の抽出
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    # 家族サイズの作成、カテゴリ化
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

    df["IsAlone"] = 1
    df.loc[df["FamilySize"] > 1, "IsAlone"] = 0

    # Age、Fareのカテゴリ化
    age_fill = df['Age'].fillna(df["Age"].median())
    df["AgeGroup"] = pd.cut(age_fill.astype(int), 5, labels=['Child', 'Young', 'Adult', 'Mid-age', 'Senior'])

    df_fill = df['Fare'].fillna(df["Fare"].median())
    df["FareGroup"] = pd.cut(df_fill, 4, labels=['Low', 'Mid', 'High', 'Very high'])

    # Ticket Frequency
    df["Ticket_Frequency"] = df.groupby("Ticket")["Ticket"].transform("count")

    # Cabin Deck
    #df["Deck"] = df["Cabin"].fillna("Ma").str[0]
    #df["Deck"] = df["Deck"].replace(["T", "G"], "M")

    return df

numerical_features = ['FamilySize', 'Ticket_Frequency']
categorical_features = ['Title', 'Sex', 'Embarked', 'AgeGroup', 'Pclass', 'IsAlone', 'FareGroup']

# 特徴量エンジニアリングの実行
train = feature_engineering(train)
test = feature_engineering(test)

# パイプラインの作成
numerical_transformer = Pipeline([
    ("imputer", KNNImputer(n_neighbors=5)),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformerの作成
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

# 前処理実行
y_train = train["Survived"]

X_train_preprocessed = preprocessor.fit_transform(train)
X_test_preprocessed = preprocessor.transform(test)

display(X_train_preprocessed.shape)
display(X_train_preprocessed[:5])

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\kuon9\AppData\Local\Temp\ipykernel_29940\2442693622.py:4: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


(891, 26)

array([[ 0.05915988, -0.57916179,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.05915988, -0.57916179,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.56097483, -0.57916179,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
       

In [18]:
# 分類器だと認識できるようにする魔法
class FixedKerasClassifier(KerasClassifier, ClassifierMixin):
    @property
    def _estimator_type(self):
        return "classifier"

# ニューラルネットワークのモデル作成関数
def create_model(units=32, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation="relu", input_shape=(X_train_preprocessed.shape[1],)),
        Dropout(dropout_rate),
        Dense(units // 2, activation="relu"),
        Dropout(dropout_rate),
        Dense(1, activation="sigmoid")
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

# early stoppingの設定
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    restore_best_weights=True
)

# KerasClassifierのラッパーの作成
nn_model = FixedKerasClassifier(
    model=create_model,
    validation_split=0.2,
    units=32,
    dropout_rate=0.3,
    learning_rate=0.001,
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]
)

# 探索するハイパーパラメータの範囲を定義
param = {
    "model__units": [16, 32, 64, 128],
    "model__dropout_rate": [0.2, 0.3, 0.4, 0.5],
    "model__learning_rate": [0.001, 0.0005],
    "batch_size": [16, 32, 64],
    "epochs": [100]
}

# RandomizedSearchCVの設定
random_search = RandomizedSearchCV(
    estimator=nn_model,
    param_distributions=param,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    verbose=0,
)

# ハイパーパラメータ探索の実行
search_result = random_search.fit(
    X_train_preprocessed,
    y_train,
    callbacks=[early_stopping]
)

Epoch 1/100


c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6526 - loss: 0.6260 - val_accuracy: 0.8067 - val_loss: 0.5274
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7474 - loss: 0.5506 - val_accuracy: 0.8571 - val_loss: 0.4606
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7684 - loss: 0.4820 - val_accuracy: 0.8235 - val_loss: 0.4145
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8084 - loss: 0.4524 - val_accuracy: 0.8403 - val_loss: 0.3948
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7979 - loss: 0.4585 - val_accuracy: 0.8487 - val_loss: 0.3879
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8105 - loss: 0.4348 - val_accuracy: 0.8571 - val_loss: 0.3763
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8084 - loss: 0.4377 - val_accuracy: 0.8655 - val_loss: 0.3720
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8147 - loss: 0.4101 - val_accuracy: 0.8655 - val_loss: 0.3

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5937 - loss: 0.6646 - val_accuracy: 0.6555 - val_loss: 0.5916
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6821 - loss: 0.5828 - val_accuracy: 0.7647 - val_loss: 0.5193
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7221 - loss: 0.5563 - val_accuracy: 0.8487 - val_loss: 0.4686
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7853 - loss: 0.5152 - val_accuracy: 0.8235 - val_loss: 0.4380
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7684 - loss: 0.4963 - val_accuracy: 0.8235 - val_loss: 0.4211
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7958 - loss: 0.4736 - val_accuracy: 0.8067 - val_loss: 0.4058
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8168 - loss: 0.4365 - val_accuracy: 0.8151 - val_loss: 0.3917
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8189 - loss: 0.4295 - val_accuracy: 0.8151 - val_loss: 0.3

...

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5453 - loss: 0.7213 - val_accuracy: 0.6639 - val_loss: 0.6309
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5874 - loss: 0.6898 - val_accuracy: 0.7227 - val_loss: 0.5819
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6295 - loss: 0.6405 - val_accuracy: 0.7983 - val_loss: 0.5552
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6589 - loss: 0.6279 - val_accuracy: 0.8067 - val_loss: 0.5336
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7074 - loss: 0.5846 - val_accuracy: 0.8067 - val_loss: 0.5058
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6737 - loss: 0.5957 - val_accuracy: 0.8151 - val_loss: 0.4915
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7347 - loss: 0.5604 - val_accuracy: 0.8151 - val_loss: 0.4752
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7158 - loss: 0.5629 - val_accuracy: 0.8235 - val_loss: 0.4

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5368 - loss: 0.7182 - val_accuracy: 0.5714 - val_loss: 0.6490
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6000 - loss: 0.6929 - val_accuracy: 0.6050 - val_loss: 0.6281
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6274 - loss: 0.6416 - val_accuracy: 0.6050 - val_loss: 0.6131
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6189 - loss: 0.6320 - val_accuracy: 0.6134 - val_loss: 0.5942
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6632 - loss: 0.5966 - val_accuracy: 0.6807 - val_loss: 0.5733
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6989 - loss: 0.5957 - val_accuracy: 0.7479 - val_loss: 0.5617
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6884 - loss: 0.6011 - val_accuracy: 0.7815 - val_loss: 0.5556
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7137 - loss: 0.5668 - val_accuracy: 0.7815 - val_loss: 0.5

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5927 - loss: 0.6707 - val_accuracy: 0.7207 - val_loss: 0.6246
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6671 - loss: 0.6339 - val_accuracy: 0.7542 - val_loss: 0.5828
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6643 - loss: 0.6135 - val_accuracy: 0.8268 - val_loss: 0.5456
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6952 - loss: 0.5865 - val_accuracy: 0.8156 - val_loss: 0.5070
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7430 - loss: 0.5506 - val_accuracy: 0.8492 - val_loss: 0.4758
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7612 - loss: 0.5208 - val_accuracy: 0.8436 - val_loss: 0.4416
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7612 - loss: 0.5256 - val_accuracy: 0.8547 - val_loss: 0.4208
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7697 - loss: 0.5048 - val_accuracy: 0.8492 - val_loss: 0.4

In [19]:
class FinalKerasWrapper(BaseEstimator, ClassifierMixin):
    # クラス変数として設定
    _estimator_type = "classifier"

    def __init__(self, estimator=None):
        self.estimator = estimator

    def fit(self, X, y, **kwargs):
        # classes_を先に設定（重要）
        self.classes_ = np.unique(y)

        # 中身のKerasモデルを学習させる
        self.estimator.fit(X, y, **kwargs)

        # 学習済みフラグを立てる
        self.n_features_in_ = X.shape[1]

        return self

    def predict(self, X, **kwargs):
        return self.estimator.predict(X, **kwargs)

    def predict_proba(self, X, **kwargs):
        return self.estimator.predict_proba(X, **kwargs)

    # この関数を追加することでsklearnが分類器として認識しやすくなる
    def _more_tags(self):
        return {'requires_y': True, 'poor_score': True}

# 最良モデルの情報を表示
print(f"Best Score: {search_result.best_score_:.4f}")
print(f"Best Params: {search_result.best_params_}")

best_model = search_result.best_estimator_
best_params = search_result.best_params_

best_nn_model = KerasClassifier(
    model=create_model,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0,
    **best_params
)

best_nn_model.fit(X_train_preprocessed, y_train)

wrapped_nn_model = FinalKerasWrapper(best_nn_model)
wrapped_nn_model.fit(X_train_preprocessed, y_train, callbacks=[early_stopping])
wrapped_nn_model.classes_ = np.array([0, 1])

print(f"Check - is_classifier(wrapped_nn_model): {is_classifier(wrapped_nn_model)}")
print(f"Check - _estimator_type: {getattr(wrapped_nn_model, '_estimator_type', 'Not Found')}")
print(f"Check - classes_: {getattr(wrapped_nn_model, 'classes_', 'Not Found')}")

Best Score: 0.8215
Best Params: {'model__units': 16, 'model__learning_rate': 0.0005, 'model__dropout_rate': 0.2, 'epochs': 100, 'batch_size': 16}


c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Check - is_classifier(wrapped_nn_model): False
Check - _estimator_type: classifier
Check - classes_: [0 1]


In [20]:
# テストデータで予測
nn_probabilities = best_model.predict_proba(X_test_preprocessed)[:, 1]
predictions = np.round(nn_probabilities).astype(int).flatten()

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [21]:
# ランダムフォレストモデルを作る
rf_param = {
    "n_estimators": randint(100, 500),
    "max_depth": [5, 10, 15, None],
    "min_samples_split": randint(2, 11),
    "min_samples_leaf": randint(1, 5)
}

rf_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=rf_param,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

rf_search.fit(X_train_preprocessed, y_train)
best_rf_model = rf_search.best_estimator_
print(f"Best RF Score: {rf_search.best_score_:.4f}")
print(f"Best RF Params: {rf_search.best_params_}")

# テストデータで予測
rf_probabilities = best_rf_model.predict_proba(X_test_preprocessed)[:, 1]

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best RF Score: 0.8361
Best RF Params: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 362}


In [23]:
# 勾配ブースティングモデルを作る
lgbm_model = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=15,
    max_depth=4,
    min_child_samples=20,
    reg_alpha=0.1,
    reg_lambda=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    verbose=-1
)

lgbm_model.fit(X_train_preprocessed, y_train)
gb_probabilities = cross_val_score(
    lgbm_model,
    X_train_preprocessed,
    y_train,
    cv=5,
    scoring='accuracy'
)

print(f"GB Score: {lgbm_model.score(X_train_preprocessed, y_train):.4f}")

predictions_lgbm = lgbm_model.predict(X_test_preprocessed)

GB Score: 0.8687


c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature nam

In [24]:
"""# アンサンブル（NNとRFとGBの平均を取る）
# 各モデルの予測確率を取得
nn_proba = best_model.predict_proba(X_test_preprocessed)
rf_proba = best_rf_model.predict_proba(X_test_preprocessed)
gb_proba = lgbm_model.predict_proba(X_test_preprocessed)

# 手動で平均を取る（soft voting の実装）
ensemble_proba = (nn_proba + rf_proba + gb_proba) / 3
predictions_voting = np.argmax(ensemble_proba, axis=1)

print(f"Ensemble predictions shape: {predictions_voting.shape}")"""

'# アンサンブル（NNとRFとGBの平均を取る）\n# 各モデルの予測確率を取得\nnn_proba = best_model.predict_proba(X_test_preprocessed)\nrf_proba = best_rf_model.predict_proba(X_test_preprocessed)\ngb_proba = lgbm_model.predict_proba(X_test_preprocessed)\n\n# 手動で平均を取る（soft voting の実装）\nensemble_proba = (nn_proba + rf_proba + gb_proba) / 3\npredictions_voting = np.argmax(ensemble_proba, axis=1)\n\nprint(f"Ensemble predictions shape: {predictions_voting.shape}")'

In [9]:
# スタッキングモデルの構築
stacking_model = StackingClassifier(
    estimators=[
        ('rf', best_rf_model),
        ('lgbm', lgbm_model),
        ('nn', wrapped_nn_model)
    ],
    final_estimator=LogisticRegression(),
    cv=5
)

stacking_model.fit(X_train_preprocessed, y_train)
predictions_stacking = stacking_model.predict(X_test_preprocessed)

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\

In [25]:
submission_ensemble = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions_stacking
})

submission_ensemble.to_csv("titanic_ensemble_solution.csv", index=False)

submission_lgbm = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions_lgbm
})

submission_lgbm.to_csv("titanic_lgbm_solution.csv", index=False)